In [ ]:
import scipy.stats as st
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import tqdm 
import tqdm.notebook
import scipy 

In [ ]:
class DatasetGenerator(object):
    def __init__(self, N=100):
        self.N = N
        self.x = None
        self.v = None
        self.refresh()
  
    def refresh(self):
        raise Exception("undefined")
    
class G1(DatasetGenerator):
    def refresh(self):
        self.x = st.uniform().rvs((self.N,2))
        self.v = st.uniform().rvs((self.N,))

class G2(DatasetGenerator):
    def refresh(self):
        self.x = st.uniform().rvs((self.N,2))
        self.v = np.exp(st.norm(-0.85, 1.3).rvs((self.N,)))

In [ ]:
# Uniform distribution

g1 = G1()
# Plot a histogram of the v array
plt.hist(g1.v, bins=30)
plt.show()

# plot the position of the points
plt.figure(figsize=(5,5))
plt.scatter(g1.x[:,0], g1.x[:,1])
plt.show()

g1.refresh() # generate a new dataset
plt.hist(g1.v, bins=30)
plt.show()

m = np.array([0., 0.])

for _ in range(10):
  g1.refresh() # refresh the dataset
  m  += 0.1*g1.x.mean()

print(m)

# Log normal distribution
g2 = G2()
plt.hist(g2.v, bins=30)
plt.show()
g2.refresh()
plt.hist(g2.v, bins=30)
plt.show()

---

### Metropolis Hastings

#### Useful functions

In [ ]:
# an optimal beta as said in the course, could be interesting to use it
def computeBeta(N0, N1, f0, f1, epsilon):
    return log(N1 / (epsilon * N0)) / (f1 - f0)

In [ ]:
def objectiveFunction(N, l, citiesV, selectedCities, pairwise_distances):
    # Compute maximum area
    max_area = np.pi / 4 * np.max(np.outer(selectedCities, selectedCities) * pairwise_distances)

    # Compute final loss
    f = np.sum(selectedCities * citiesV) - l * N * max_area
    return -f

# Old code
# def maxArea(citiesX, selectedCities):
#     n = citiesX.shape[0]
#     result = np.zeros((n,n))
#     for i in range(n):
#         for j in range(n):
#             if selectedCities[i]==1 and selectedCities[j]==1:
#                 result[i,j]=(citiesX[i,0]-citiesX[j,0])*(citiesX[i,0]-citiesX[j,0])+(citiesX[i,1]-citiesX[j,1])*(citiesX[i,1]-citiesX[j,1])
#     maxR = np.amax(result)
#     return np.pi * maxR / 4
# def objectiveFunction(N,l,citiesV,selectedCities, _): 
#     f = 0
#     for i in range(N):
#         if selectedCities[i]:
#             f += citiesV[i]
#     f = f - l * N * maxArea(citiesX, selectedCities)
#     return -f
    

In [ ]:
def acceptancePb(selectedCities_i,selectedCities_j,beta,N,l,citiesV, pairwise_distances):
    fi = objectiveFunction(N, l, citiesV, selectedCities_i, pairwise_distances)
    fj = objectiveFunction(N, l, citiesV, selectedCities_j, pairwise_distances)
    result = np.exp(-beta * (fj - fi))
    return min(1, result)

In [ ]:
def pbij(selectedCities_i, selectedCities_j, beta, N, l, citiesV):
    # not used in the algorithm, could be useful to plot and compare statistics
    if selectedCities_i == selectedCities_j:
        s = 0
        for k in range (N):
            selectedCities_k=np.copy(selectedCities_i)
            selectedCities_k[k]=1-selectedCities_i[k]
            a_ik=acceptancePb(selectedCities_i,selectedCities_k,beta,n,l,citiesV)
            phi_ik=(1/2)*(1/N)
            s+=phi_ik*a_ik
        return 1-s
    else:
        a_ij=acceptancePb(selectedCities_i,selectedCities_j,beta,n,l,citiesV)
        phi_ij=(1/2)*(1/N) #pb of chosing a city*pb choosing if 0 or 1
        return a_ij*phi_ij
    

In [ ]:
def plotStatistics(): #useful to see the evolution of the number of cities selected and the f function
    return

#### The Algorithm

**Algorithm:**
1. Start from a random distribution
2. Select a city uniformly at random
3. choose if city had to be in the set, meaning selectedCities should be set to 0 or 1 (pb 1/2)
4. Accept or not that choice for that city, using the acceptance pb
5. Repeat (for a fixed number of times m)

Here is step 2,3,4:

In [ ]:
# Step 2,3,4
def step(N, citiesX, citiesV, selectedCities_i, beta, l, pairwise_distances):
    k = np.random.randint(0, N);
    if np.random.rand() < 0.5: # Remove a city
        if selectedCities_i[k] == 0: # same state then before
            return selectedCities_i # do nothing, it is accepted
        else:
            selectedCities_k = np.copy(selectedCities_i)
            selectedCities_k[k] = 0 # city removed from set
            a_ik = acceptancePb(selectedCities_i, selectedCities_k, beta, N, l, citiesV, pairwise_distances) 
            if np.random.rand() < a_ik:
                return selectedCities_k #accepted!
            else:
                return selectedCities_i #refused
    else: # Add a city
        if selectedCities_i[k] == 1: # do nothing, city already in set
            return selectedCities_i
        else:
            selectedCities_k = np.copy(selectedCities_i)
            selectedCities_k[k] = 1 # add city to set
            #could of course be computed in a smarter way
            a_ik = acceptancePb(selectedCities_i, selectedCities_k, beta, N, l, citiesV, pairwise_distances)
            if np.random.rand() < a_ik:
                return selectedCities_k #city added!
            else:
                return selectedCities_i #refused


New run

In [ ]:
np.random.seed(42)
n_iter = 2000

# parametersc
l = 0.3 # lambda in [0,1]
beta = 100

# initialization
N = 100
n_selected = 0
g = G1(N)
g = G2(N)
citiesX = g.x
citiesV = g.v

selectedCities_n = np.random.randint(2, size=(N))

# Precompute all pairwise distances between points
# TODO: This probably should be euclidean distance instead of squared distance?
# (this currently matches Heloise's original code)
pairwise_distances = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(citiesX, 'sqeuclidean'))
    
fs = np.zeros(n_iter) #keep record of objective function (in fact, minus objective function)
fs[0] = objectiveFunction(N, l, citiesV, selectedCities_n, pairwise_distances)

for m in tqdm.notebook.tqdm(range(n_iter)):
    fs[m] = objectiveFunction(N, l, citiesV, selectedCities_n, pairwise_distances)
    selectedCities_n = step(N, citiesX, citiesV, selectedCities_n, beta, l, pairwise_distances)
    
print(selectedCities_n)
print('fM '+ str(fs[n_iter-1]))
plt.figure()
plt.plot(np.arange(n_iter), fs);

**Discussion:**
* convergence in terms of beta ?
* Choice of beta? Small then large, or use the optimal beta of the course (need precomputation?)
* Run the simulation many times? Then pick the min? Or do simulated annealing (with different beta)

**To be added:**
* statistics and visualization (of the cities at their location, with a color corresponding to selected or not)
